# Identifying Features Associated with Groups and First Model Attempts

## Load Data

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# columns of interest
cols_to_extract = ['imonth', 'iyear','iday',
 'country_txt','gname','attacktype1_txt',
 'success','suicide',
 'weaptype1_txt','weapsubtype1_txt',
 'targtype1_txt','targsubtype1_txt',
 'individual','nperps','claimed',
 'nkill','nwound',
 'property','propextent_txt',
 'ishostkid','nhostkid','hostkidoutcome_txt','ransom']

# Load data
df = pd.read_excel('GTD_0617dist/globalterrorismdb_0617dist.xlsx',
                   usecols=cols_to_extract,
                   na_values = ['Unknown','-99','-9','Not Applicable'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170350 entries, 0 to 170349
Data columns (total 23 columns):
iyear                 170350 non-null int64
imonth                170350 non-null int64
iday                  170350 non-null int64
country_txt           170350 non-null object
success               170350 non-null int64
suicide               170350 non-null int64
attacktype1_txt       163925 non-null object
targtype1_txt         165477 non-null object
targsubtype1_txt      161005 non-null object
gname                 92044 non-null object
individual            170350 non-null int64
nperps                26173 non-null float64
claimed               102742 non-null float64
weaptype1_txt         156498 non-null object
weapsubtype1_txt      150924 non-null object
nkill                 160668 non-null float64
nwound                155025 non-null float64
property              170350 non-null int64
propextent_txt        41479 non-null object
ishostkid             169903 non-null fl

## Extract events associated with groups of interest

In [2]:
# remove events with no group affiliation
no_grp = df.gname.isnull() | df.individual
with_grp = ~no_grp
df = df[with_grp]

# don't need the 'individual' column any more
df.drop('individual',axis=1,inplace=True)

# only keep the top n groups with the most incidents
n_groups = 50 #for all groups: df['gname'].nunique()

top_grps = df['gname'].value_counts().head(n_groups).index
df = df[df.gname.isin(top_grps)]

print('Number of events affiliated with individuals or unknown group: ',sum(no_grp))
print('Number of events affiliated with a group: ',sum(with_grp))
print('Number of events affiliated with top {} groups: {}'.format(n_groups,len(df)))

Number of events affiliated with individuals or unknown group:  78620
Number of events affiliated with a group:  91730
Number of events affiliated with top 50 groups: 60221


## Modify some unwanted columns and values

In [3]:
# replace some values not correctly dealt with by pandas import
df.replace(-9,np.nan,inplace=True)
df.replace(-99,np.nan,inplace=True)

# entries without month or day - treat as 1st January
df['imonth'].replace(0,1,inplace=True)
df['iday'].replace(0,1,inplace=True)

# create a date column, then get rid of the month and day columns
df['date']=pd.to_datetime(dict(year=df.iyear, month=df.imonth, day=df.iday)) 
df.rename(columns={'iyear':'year'}, inplace=True)
df.drop(['imonth','iday'],axis=1,inplace=True)

# set date as the index
df.set_index('date',inplace=True)

# If no claimed info - treat as not claimed
df['claimed'].fillna(0,inplace=True)

# remove some values that don't give useful information
df['weaptype1_txt'].replace('Other',np.nan,inplace=True)

df['weapsubtype1_txt'].replace(['Unknown Gun Type', 'Unknown Explosive Type',
                                'Other Explosive Type', 'Unknown Weapon Type',
                                'Other Gun Type'], np.nan, inplace=True)

df['targtype1_txt'].replace('Other',np.nan,inplace=True)

df['targsubtype1_txt'].replace(['Other Personnel', 'Other (including online news agencies)', 'Other Facility'],
                               np.nan,inplace=True)


df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60221 entries, 1970-01-31 to 2016-12-31
Data columns (total 20 columns):
year                  60221 non-null int64
country_txt           60221 non-null object
success               60221 non-null int64
suicide               60221 non-null int64
attacktype1_txt       56890 non-null object
targtype1_txt         59024 non-null object
targsubtype1_txt      57737 non-null object
gname                 60221 non-null object
nperps                10123 non-null float64
claimed               60221 non-null float64
weaptype1_txt         53851 non-null object
weapsubtype1_txt      27327 non-null object
nkill                 55491 non-null float64
nwound                52350 non-null float64
property              53930 non-null float64
propextent_txt        14450 non-null object
ishostkid             60070 non-null float64
nhostkid              4991 non-null float64
ransom                29995 non-null float64
hostkidoutcome_txt    3166 non-nul

## Categorise some columns to reduce no. features

In [4]:
# Numeric columns - convert to values for 0, 1, 2-10, and more than 10 
conv_numeric = ['nkill','nwound','nperps','nhostkid']

for col in conv_numeric:
    df[col] = pd.cut(df[col],
                        [-0.1,0.9,1.9,10.9,max(df[col])+0.1],
                        labels=['0_'+col,'1_'+col,'2to10_'+col,'11+_'+col])

# boolean columns - add column suffix
conv_bool = ['success','suicide','claimed','ishostkid','ransom','property']

for col in conv_bool:
    df[col].replace({0:('0_'+col),1:('1_'+col)},inplace=True)

# bin year in to decades
df['year'] = pd.cut(df['year'],
                        [1969.9,1979.9,1989.9,1999.9,2009.9,2019.9],
                        labels=['1970s_yr','1980s_yr','1990s_yr','2000s_yr','2010s_yr'])  

# add suffixes to some column values to help understanding later
add_suffix = {'targtype1_txt':'_target',
              'attacktype1_txt':'_attack', 
              'targsubtype1_txt':'_targetsub',
              'weaptype1_txt':'_weapon',
              'weapsubtype1_txt':'_weaponsub',
              'propextent_txt':'_property'}

for col,suffix in add_suffix.items():
    rows = df[col].notnull()
    df.loc[rows,col] = df.loc[rows,col] + suffix

display(df.head())

# warning message below r.e. empty bins

C:\Users\jackr\Anaconda3\lib\site-packages\pandas\core\reshape\tile.py:130: RuntimeWarning: invalid value encountered in less
  if (np.diff(bins) < 0).any():


,year,country_txt,success,suicide,attacktype1_txt,targtype1_txt,targsubtype1_txt,gname,nperps,claimed,weaptype1_txt,weapsubtype1_txt,nkill,nwound,property,propextent_txt,ishostkid,nhostkid,ransom,hostkidoutcome_txt
date,,,,,,,,,,,,,,,,,,,,
1970-01-31,1970s_yr,Philippines,0_success,0_suicide,NaN,Military_target,"Military Personnel (soldiers, troops, officers...",New People's Army (NPA),NaN,0_claimed,NaN,NaN,0_nkill,1_nwound,0_property,NaN,0_ishostkid,NaN,0_ransom,NaN
1970-04-02,1970s_yr,Philippines,1_success,0_suicide,NaN,Military_target,"Military Personnel (soldiers, troops, officers...",New People's Army (NPA),NaN,0_claimed,NaN,NaN,2to10_nkill,0_nwound,0_property,NaN,0_ishostkid,NaN,0_ransom,NaN
1970-04-25,1970s_yr,Turkey,1_success,0_suicide,Bombing/Explosion_attack,Airports & Aircraft_target,Airline Officer/Personnel_targetsub,Palestinians,NaN,0_claimed,Explosives/Bombs/Dynamite_weapon,NaN,0_nkill,0_nwound,1_property,NaN,0_ishostkid,NaN,0_ransom,NaN
1970-05-04,1970s_yr,Paraguay,1_success,0_suicide,Assassination_attack,Government (Diplomatic)_target,"Diplomatic Personnel (outside of embassy, cons...",Palestinians,2to10_nperps,0_claimed,Firearms_weapon,Automatic Weapon_weaponsub,1_nkill,1_nwound,0_property,NaN,0_ishostkid,NaN,0_ransom,NaN
1970-06-26,1970s_yr,United Kingdom,0_success,0_suicide,Armed Assault_attack,Terrorists/Non-State Militia_target,Terrorist_targetsub,Irish Republican Army (IRA),NaN,0_claimed,Incendiary_weapon,NaN,2to10_nkill,NaN,1_property,NaN,0_ishostkid,NaN,0_ransom,NaN


## Calculate Characterising Values for each Group
i.e. the feature values that are most characteristic of an attack by each group

In [5]:
# no. incidents associated with each group
grp_incs = df['gname'].value_counts()

# loop over all columns excluding gname
grp_weights = dict()

for col in df.drop('gname',axis=1).columns:
    # for each group, how many times each unique value appears in this column
    grp_cnts = df.groupby('gname')[col].value_counts().unstack(col,fill_value=0)
    
    # for each unique value in this column, count how many groups have an incident including it
    # convert this for tf-idf weight using log(n_groups/count)
    w_col = np.log(n_groups/(grp_cnts>0).sum())
    
    # multiply w_col by no. occurences each column value to get weight for each group
    # normalise by no. incidents for that group, so groups can be compared more easily
    grp_weights[col] = (grp_cnts*w_col).div(grp_incs,axis=0)

# merge unique values for each column in to one large data frame
grp_aw = pd.DataFrame(index=top_grps)
for key, w_col in grp_weights.items():
    grp_aw = pd.merge(grp_aw, w_col, left_index=True, right_index=True,how='outer')
    
grp_aw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, Abu Sayyaf Group (ASG) to United Liberation Front of Assam (ULFA)
Columns: 312 entries, 1970s_yr to Successful Rescue
dtypes: float64(312)
memory usage: 124.8+ KB


## Print Top n Features for Top m Groups

In [149]:
n_print_grp=10
n_print_feat=5

for grp in grp_incs.head(n_print_grp).index:    
    print(grp_aw.loc[grp].sort_values(ascending=False).head(n_print_feat))
    print('----------------------------------------------')

Afghanistan       2.791160
2010s_yr          0.271797
NATO_targetsub    0.136084
1_claimed         0.133650
1_suicide         0.067364
Name: Taliban, dtype: float64
----------------------------------------------
Peru                      1.826944
1980s_yr                  0.340959
1990s_yr                  0.090445
Electricity_targetsub     0.030300
Dynamite/TNT_weaponsub    0.025588
Name: Shining Path (SL), dtype: float64
----------------------------------------------
Iraq         2.141596
2010s_yr     0.328504
Syria        0.232947
1_suicide    0.161201
1_claimed    0.110172
Name: Islamic State of Iraq and the Levant (ISIL), dtype: float64
----------------------------------------------
El Salvador              3.198704
1980s_yr                 0.380603
Electricity_targetsub    0.068140
1990s_yr                 0.066858
Utilities_target         0.035210
Name: Farabundo Marti National Liberation Front (FMLN), dtype: float64
----------------------------------------------
Somalia      2.

## Create Dummy variables for Each Category Value

In [182]:
df_noind=df.reset_index(drop=True)

# labels
y = pd.get_dummies(df_noind['gname'])
display(y.head())

# features
X = pd.get_dummies(df_noind.drop('gname',axis=1)) 
display(X.head())

,Abu Sayyaf Group (ASG),African National Congress (South Africa),Al-Qaida in Iraq,Al-Qaida in the Arabian Peninsula (AQAP),Al-Shabaab,Algerian Islamic Extremists,Bangsamoro Islamic Freedom Movement (BIFM),Basque Fatherland and Freedom (ETA),Boko Haram,Chechen Rebels,...,Revolutionary Armed Forces of Colombia (FARC),Separatists,Shining Path (SL),Sikh Extremists,Sinai Province of the Islamic State,Taliban,Tehrik-i-Taliban Pakistan (TTP),Tripoli Province of the Islamic State,Tupac Amaru Revolutionary Movement (MRTA),United Liberation Front of Assam (ULFA)
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,year_1970s_yr,year_1980s_yr,year_1990s_yr,year_2000s_yr,year_2010s_yr,country_txt_Afghanistan,country_txt_Algeria,country_txt_Angola,country_txt_Argentina,country_txt_Austria,...,nhostkid_2to10_nhostkid,nhostkid_11+_nhostkid,ransom_0_ransom,ransom_1_ransom,hostkidoutcome_txt_Attempted Rescue,hostkidoutcome_txt_Combination,hostkidoutcome_txt_Hostage(s) escaped (not during rescue attempt),hostkidoutcome_txt_Hostage(s) killed (not during rescue attempt),hostkidoutcome_txt_Hostage(s) released by perpetrators,hostkidoutcome_txt_Successful Rescue
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


## Fit a Classifier to the Data

In [16]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import average_precision_score, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42)
model=OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)

y_pred = model.predict(X_test)

print('Accuracy score:',accuracy_score(y_test,y_pred))
print('Precision score:',average_precision_score(y_test,y_pred))

0.85775170199811812

## Investigate the Results of the Classifier

In [155]:
# extract group for each event in test data
test_labels = y_test.stack()
test_labels = test_labels[test_labels>0].index.get_level_values(1)

# extract prediction for each event in test data
pred_labels = y_test.columns[y_pred.argmax(axis=1)]

# create a data frame of labels and predictions
labels = pd.DataFrame({'true':test_labels.values, 'pred':pred_labels.values})

# was the predcition correct?
labels['correct'] = labels['pred']==labels['true']

# labelled correctly / total events
frac_true = (labels.loc[labels.correct,'true'].value_counts()/labels['true'].value_counts()).sort_values(ascending=False)

# predicted correctly / predicted total
frac_pred = (labels.loc[labels.correct,'pred'].value_counts()/labels['pred'].value_counts()).sort_values(ascending=False)


print('-------------------------------------------')
print('Groups commonly labelled incorrectly:')
print('-------------------------------------------')
display(labels.loc[~labels.correct,'true'].value_counts().head(5))
display(frac_true.tail(5).sort_values())
print('-------------------------------------------')
print('Common incorrect predictions:')
print('-------------------------------------------')
display(labels.loc[~labels.correct,'pred'].value_counts().head(5))
display(frac_pred.tail(5).sort_values())
print('-------------------------------------------')
print('Groups commonly labelled correctly:')
print('-------------------------------------------')
display(labels.loc[labels.correct,'true'].value_counts().head(5))
display(frac_true.head(5))
print('-------------------------------------------')
print('Common correct predictions:')
print('-------------------------------------------')
display(labels.loc[labels.correct,'pred'].value_counts().head(5))
display(frac_pred.head(5))

-------------------------------------------
Groups commonly labelled incorrectly:
-------------------------------------------


National Liberation Army of Colombia (ELN)       281
Revolutionary Armed Forces of Colombia (FARC)    228
Tupac Amaru Revolutionary Movement (MRTA)        157
Maoists                                          135
Al-Qaida in Iraq                                 123
Name: true, dtype: int64

Tupac Amaru Revolutionary Movement (MRTA)     0.122905
Muslim extremists                             0.253247
Al-Qaida in Iraq                              0.305085
National Liberation Army of Colombia (ELN)    0.352535
United Liberation Front of Assam (ULFA)       0.363636
Name: true, dtype: float64

-------------------------------------------
Common incorrect predictions:
-------------------------------------------


Abu Sayyaf Group (ASG)                            831
Shining Path (SL)                                 165
Islamic State of Iraq and the Levant (ISIL)       153
Revolutionary Armed Forces of Colombia (FARC)     142
Communist Party of India - Maoist (CPI-Maoist)    138
Name: pred, dtype: int64

...

Taliban                                             2002
Shining Path (SL)                                   1338
Islamic State of Iraq and the Levant (ISIL)         1278
Farabundo Marti National Liberation Front (FMLN)     992
Al-Shabaab                                           795
Name: true, dtype: int64

Donetsk People's Republic                                      1.000000
Al-Shabaab                                                     1.000000
Nicaraguan Democratic Force (FDN)                              0.996350
Taliban                                                        0.992071
National Union for the Total Independence of Angola (UNITA)    0.991667
Name: true, dtype: float64

-------------------------------------------
Common correct predictions:
-------------------------------------------


Taliban                                             2002
Shining Path (SL)                                   1338
Islamic State of Iraq and the Levant (ISIL)         1278
Farabundo Marti National Liberation Front (FMLN)     992
Al-Shabaab                                           795
Name: pred, dtype: int64

Algerian Islamic Extremists                 1.000000
Donetsk People's Republic                   1.000000
African National Congress (South Africa)    1.000000
Taliban                                     0.998504
Nicaraguan Democratic Force (FDN)           0.996350
Name: pred, dtype: float64

## Abu Sayyaf Group (ASG): Frequently Predicted Wrongly

A look at some of the features of ASG events, and the groups that are often mistaken for ASG

In [161]:
print('=============================')
print('Abu Sayyaf Group (ASG)')
print('=============================')
asg=df[df.gname=='Abu Sayyaf Group (ASG)']
display(asg['country_txt'].value_counts())

print('=============================')
print('Philippines')
print('=============================')
print(df[df.country_txt == 'Philippines']['gname'].value_counts())

print('=============================')
print('Malaysia')
print('=============================')
print(df[df.country_txt == 'Malaysia']['gname'].value_counts())

print('================================================')
print('Regularly Mistaken for Abu Sayyaf Group (ASG)')
print('================================================')
print(labels.loc[(~labels.correct) & (labels.pred=='Abu Sayyaf Group (ASG)')].true.value_counts().head(10))

asg_w = grp_aw.loc['Abu Sayyaf Group (ASG)']
farc_w = grp_aw.loc['Revolutionary Armed Forces of Colombia (FARC)']
asgfarc = pd.DataFrame({'Abu Sayyaf Group (ASG)':asg_w, 'Revolutionary Armed Forces of Colombia (FARC)':farc_w})
asgfarc['diff'] = asgfarc['Abu Sayyaf Group (ASG)']-asgfarc['Revolutionary Armed Forces of Colombia (FARC)']

print('================================================')
print('Features More Common in ASG than FARC')
print('================================================')
display(asgfarc.sort_values('diff').tail(5).sort_values(by='diff',ascending=False))

print('=========================================================================')
print('Features More Common FARC than ASG')
print('=========================================================================')
display(asgfarc.sort_values('diff').head(5))

print('=========================================================================')
print('Features Similar in Both')
print('=========================================================================')
inboth = (asgfarc['Abu Sayyaf Group (ASG)']>0.005) & (asgfarc['Revolutionary Armed Forces of Colombia (FARC)']>0.005)
display(abs(asgfarc.loc[inboth]).sort_values('diff'))


Abu Sayyaf Group (ASG)


Philippines    451
Malaysia        19
Name: country_txt, dtype: int64

Philippines
New People's Army (NPA)                        2412
Abu Sayyaf Group (ASG)                          451
Moro Islamic Liberation Front (MILF)            363
Bangsamoro Islamic Freedom Movement (BIFM)      320
Islamic State of Iraq and the Levant (ISIL)       8
Muslim extremists                                 8
Death Squad                                       2
Name: gname, dtype: int64
Malaysia
Abu Sayyaf Group (ASG)                         19
Islamic State of Iraq and the Levant (ISIL)     1
Name: gname, dtype: int64
Regularly Mistaken for Abu Sayyaf Group (ASG)
Revolutionary Armed Forces of Colombia (FARC)     137
National Liberation Army of Colombia (ELN)        135
M-19 (Movement of April 19)                        63
New People's Army (NPA)                            49
Narco-Terrorists                                   44
Bangsamoro Islamic Freedom Movement (BIFM)         34
Muslim extremists                                  33
Communist Party of India - Maoist (CPI-

,Abu Sayyaf Group (ASG),Revolutionary Armed Forces of Colombia (FARC),diff
Philippines,1.886632,0.000000,1.886632
Malaysia,0.130125,0.000000,0.130125
2010s_yr,0.212479,0.084609,0.127871
Commercial Maritime_targetsub,0.058214,0.000000,0.058214
1_ransom,0.049625,0.007018,0.042607


Features More Common FARC than ASG


,Abu Sayyaf Group (ASG),Revolutionary Armed Forces of Colombia (FARC),diff
Colombia,0.000000,1.818547,-1.818547
1980s_yr,0.000000,0.098844,-0.098844
1990s_yr,0.031453,0.083417,-0.051964
1970s_yr,0.000000,0.041340,-0.041340
Oil_targetsub,0.000000,0.028323,-0.028323


Features Similar in Both


,Abu Sayyaf Group (ASG),Revolutionary Armed Forces of Colombia (FARC),diff
Farm/Ranch_targetsub,0.013110,0.016889,0.003778
2000s_yr,0.099288,0.092180,0.007108
Hostage Taking (Kidnapping)_attack,0.013897,0.005348,0.008549
1_claimed,0.022189,0.006858,0.015330
1_ransom,0.049625,0.007018,0.042607
1990s_yr,0.031453,0.083417,0.051964
2010s_yr,0.212479,0.084609,0.127871


## Countries of ASG Predicted Events

ASG often predicted for events in countries they were never active in. Suggests country should be weighted much more heavily?

In [255]:
# extract the test set events from the full df created earlier (one using same indexing)
test_events = df_noind.iloc[y_test.index]

# merge the predicted labels columns to the test_events df
test_events = pd.merge(test_events, 
                       pd.DataFrame({'pred':pred_labels},index=test_events.index),
                       left_index=True,right_index=True)

test_events['true'] = test_events['gname']==test_events['pred']

print('======================================================================')
print('Countries of Events Incorrectly Predicted as Abu Sayyaf Group (ASG)')
print('======================================================================')
display(test_events[(test_events.pred=='Abu Sayyaf Group (ASG)') & (~test_events.true)]['country_txt'].unique())
print('======================================================================')
print('Countries Where Abu Sayyaf Group (ASG) Carried Out Attacks')
print('======================================================================')
df_noind[df_noind.gname=='Abu Sayyaf Group (ASG)']['country_txt'].unique()

Countries of Events Incorrectly Predicted as Abu Sayyaf Group (ASG)


array(['Colombia', 'Panama', 'Philippines', 'Yemen', 'France', 'India',
       'Algeria', 'Libya', 'Israel', 'United Kingdom', 'Turkey', 'Russia',
       'Peru', 'Ecuador', 'Sri Lanka', 'Central African Republic',
       'West Bank and Gaza Strip', 'Greece', 'Brazil', 'Egypt',
       'El Salvador', 'Zambia', 'Saudi Arabia', 'Honduras', 'Nigeria',
       'Bangladesh', 'Italy', 'Austria', 'Iraq', 'Kuwait', 'Belgium',
       'South Sudan', 'Guatemala', 'Canada', 'Denmark', 'Guadeloupe',
       'Netherlands', 'Lebanon', 'Bulgaria', 'Tunisia', 'Iran', 'Pakistan',
       'Mali', 'Bolivia', 'Chile', 'Georgia', 'Syria', 'Costa Rica',
       'Kosovo', 'South Yemen', 'Chad', 'Botswana'], dtype=object)

Countries Where Abu Sayyaf Group (ASG) Carried Out Attacks


array(['Philippines', 'Malaysia'], dtype=object)